In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, Trainer, TrainingArguments
from datasets import load_dataset

from captum.attr import IntegratedGradients

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# replace <PATH-TO-SAVED-MODEL> with the real path of the saved model
model_path = 'bert-base-uncased'

# load model
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.start_logits, output.end_logits

def forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    return pred.max(1).values

In [4]:
# Fine-tuning HuggingFace pre-trained model on HuggingFace SQUAD dataset
model_name = 'bert-base-uncased'
dataset_name = 'sst'

model = BertForSequenceClassification.from_pretrained(model_name, from_tf=True)
tokenizer = BertTokenizer.from_pretrained(model_name)
dataset = load_dataset(dataset_name, 'default')

Downloading: 100%|██████████| 536M/536M [02:49<00:00, 3.16MB/s]
Loading a TensorFlow model in PyTorch, requires both PyTorch and TensorFlow to be installed. Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation instructions.
Loading a TensorFlow model in PyTorch, requires both PyTorch and TensorFlow to be installed. Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation instructions.


ModuleNotFoundError: No module named 'tensorflow'

In [14]:
model.load_state_dict(torch.load('BERTckpt.ckpt'))

RuntimeError: Error(s) in loading state_dict for BertForSequenceClassification:
	Missing key(s) in state_dict: "bert.embeddings.position_ids", "bert.embeddings.word_embeddings.weight", "bert.embeddings.position_embeddings.weight", "bert.embeddings.token_type_embeddings.weight", "bert.embeddings.LayerNorm.weight", "bert.embeddings.LayerNorm.bias", "bert.encoder.layer.0.attention.self.query.weight", "bert.encoder.layer.0.attention.self.query.bias", "bert.encoder.layer.0.attention.self.key.weight", "bert.encoder.layer.0.attention.self.key.bias", "bert.encoder.layer.0.attention.self.value.weight", "bert.encoder.layer.0.attention.self.value.bias", "bert.encoder.layer.0.attention.output.dense.weight", "bert.encoder.layer.0.attention.output.dense.bias", "bert.encoder.layer.0.attention.output.LayerNorm.weight", "bert.encoder.layer.0.attention.output.LayerNorm.bias", "bert.encoder.layer.0.intermediate.dense.weight", "bert.encoder.layer.0.intermediate.dense.bias", "bert.encoder.layer.0.output.de

In [4]:
train_dataset = dataset['train']
eval_dataset = dataset['validation']

In [5]:
train_dataset = train_dataset.map(lambda batch: tokenizer(batch['context'], batch['question'], truncation=True, padding='max_length'))
eval_dataset = eval_dataset.map(lambda batch: tokenizer(batch['context'], batch['question'], truncation=True, padding='max_length'))

Loading cached processed dataset at C:\Users\jzcha\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\de2e67b822b2ef3f4b137148d0758f48075e3892c359c50271ef6c9add7e794a\cache-4777ffb640144f40.arrow
Loading cached processed dataset at C:\Users\jzcha\.cache\huggingface\datasets\squad_v2\squad_v2\2.0.0\de2e67b822b2ef3f4b137148d0758f48075e3892c359c50271ef6c9add7e794a\cache-d0b1c0c66f366e84.arrow


In [6]:
training_args = TrainingArguments(
    output_dir = "./saves",
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 2,
    learning_rate = 5e-05,
    weight_decay = 1,
    num_train_epochs = 5,
)
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset)

In [7]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: title, context, question, id, answers.
***** Running training *****
  Num examples = 130319
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 325795
  0%|          | 0/325795 [00:00<?, ?it/s]

KeyError: 'loss'